# 선형 회귀 학습 심화
## 하이퍼 파라미터 튜닝(GridSearchCV)
- 가장 높은 성능으로 학습하게 하는 하이퍼 파라미터 탐색 필요
- learning rate 및 정규화 정도에 따라 성능 차이 존재
## 데이터 전처리
## 파이프라인

## 검증 데이터셋 + for문

In [5]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [6]:
l_eta0 = [0.0001, 0.001, 0.01, 0.1]
l_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

In [7]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1234)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.25,
                                                  random_state=1234)
best_R2 = 0
for eta0 in l_eta0:
    for alpha in l_alpha:
        reg = SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', eta0=eta0, alpha=alpha, random_state=1234)
        reg.fit(X_val, y_val)
        current_R2 = reg.score(X_val, y_val)
        if current_R2 > best_R2:
            best_R2 = current_R2
            best_eta0 = eta0
            best_alpha = alpha
reg = SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', eta0=best_eta0, alpha=best_alpha, random_state=1234)
reg = reg.fit(X_train, y_train)
test_R2 = reg.score(X_test, y_test)

print('최적 러닝레이트:',best_eta0)
print('최적 정규화 계수:',best_alpha)
print('검증 데이터셋 R2:',best_R2)
print('테스트 데이터셋 R2:',test_R2)


최적 러닝레이트: 0.0001
최적 정규화 계수: 0.0001
검증 데이터셋 R2: 0.5521356829103785
테스트 데이터셋 R2: 0.4731219201091438


## GridSearchCV(Cross Validation)
- 교차 검증 활용: 검증 데이터셋을 여러 조합으로 평가한 후 score 평균
- KFold 함수: 교차 검증을 위해 사용, validator 객체 생성
- e.g. 라쏘 회귀에서 어떤 alpha 값이 가장 좋은지, 어떤 eta0 값이 가장 좋은지 -> 총 28가지의 경우 중 어떤 조합이 가장 좋은지

### GrideSerchCV 객체로 fit을 할 시 KFold 교차 검증 + for문 반복 실험으로 최적의 파라미터 조합을 찾아서 fit

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1234)

# 중요!!
param_grid = {'alpha': l_alpha, 'eta0': l_eta0}
# 회귀 모델 객체가 estimator에 들어감
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)
grid_search = GridSearchCV(estimator=SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', random_state=1234), param_grid=param_grid, cv=kfold)